In [4]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper



#### First tool Wikipedia

In [16]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1 , doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [6]:
wiki.name

'wikipedia'

In [12]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
RecursiveCharacterTextSplitter(chunk_size = 1000 , chunk_overlap = 200).split_documents(docs)


db = FAISS.from_documents(
    documents=docs[:30],
     embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    
)

retreiver = db.as_retriever()
retreiver


OllamaEmbeddings: 100%|██████████| 1/1 [00:12<00:00, 12.12s/it]


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002312A4C1C50>)

#### Second tool LANGSMITH

In [13]:
from langchain.tools.retriever import create_retriever_tool

retreiver_tool = create_retriever_tool(retreiver , "langsmith_search" ,
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [14]:
retreiver_tool.name

'langsmith_search'

##### Third Tool Arxiv

In [15]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
 

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1 , doc_content_chars_max=200)
arxiv =ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [17]:
tools = [wiki , arxiv , retreiver_tool]

In [18]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\PMLS\\Desktop\\01_Translate_Video\\TransVid\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002311398E660>, retriever=VectorStoreRetriever(tags=['FAISS', 'Ollam

##### Agents

In [42]:
from langchain.llms import Ollama

llm = Ollama(model = "phi3")

In [43]:

from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [44]:
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm,tools, prompt)

In [45]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent , tools=tools , verboe= True)

agent_executor

AgentExecutor(agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d

In [ ]:
agent_executor.invoke({"input":"Tell me about Langsmith"})


In [ ]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})

